# 1. 免学部分
* p137~p138
* 扣除免学部分只有7面


# 2. 前言

* 大家在做题目前一定要先把第7章的书读一下
* 写本作业，请使用 Anaconda 里的 Notebook
* 我做了一个[打包下载 Link](https://downgit.github.io/#/home?url=https://github.com/liuhui998/pe2/tree/master/07-data-type), 大家下载下来解压后，使用 Notebook 打开

# 3. 学习概要
* 我会在每道题有前标明对的书页码
* 如果超纲我会提前提示，并加上资料 Link
---
重点知识：

* DataFrame 对象里的某一列是 object(对象), 里面可能有字符串也会有数字
---
知识点整理

* astype 将 DataFrame 对象里的某一列转换为指定类型
* dropna 将 DataFrame 对象里为空的数据删掉 [dropna滤除缺失数据](https://blog.csdn.net/weixin_38168620/article/details/79596798)
* [pd.read_csv/pd.read_excel 函数中 na_values 参数的意义和用法](https://blog.csdn.net/weixin_44520259/article/details/106053987)
* [pd.to_numeric 详细的用法](https://blog.csdn.net/jinruoyanxu/article/details/79150896)

In [2]:
import pandas as pd

# 4. 处何为北美的大客户生成唯一标号
--- 
备注：这里综合了 p100/p109/p132面的内容

* 现在回过头讲第5章的内容，是为了加深大家的理解，因为知识是交叉的

  了解 [na_values 的意义和用法](https://blog.csdn.net/weixin_44520259/article/details/106053987)，后面清洗数据上会轻松很多 
  
* dropna 是删除 DataFrame 里为空的数据  [dropna滤除缺失数据](https://blog.csdn.net/weixin_38168620/article/details/79596798)

* astype 很好理解，但是 DataFrame 里的某一列是 object(对象), 里面可能有字符串也会有数字

---
背景：

* 你现在负责追踪公司北美大客户的情况
* 你需要为每个一个客户生成一个唯一编号
* 编号为 区域(region) + 国家(country) + 卖家名（seller_name)
* 你要把把所有空的数据去掉(比如没有区域、卖家名信息的行）
* 同时也要解决 NA(North America) 被 Pandas 误为空值的情况
---
*  读取 whole_seller_stats.xlsx 到stats

   在 pd.read_excel 函数中指定 na_values = [''], keep_default_na = False 这两个参数
   
   na_values = [''] 表示只把 ‘’ (空值) 当成空值/缺失值
   
   keep_default_na = False 表示不把 NA 等字符串当成空值 

* 把 stats 空的数据去掉(比如没有区域、卖家名信息的行), 放到 stats_new 中

* 查看 stats_new 里各列的数据类型  

* 查看 seller_name == "John" 这一行 country 的数据类型 (第3行，第2列）

* 查看 seller_name == "Tom" 这一行 country 的数据类型 (第2行，第2列）

* 请尝试执行下面代码，看有什么问题

```
 stats_new['region'] + "_" + stats_new['country'] + "_" + stats['seller_name']
```

* 请将 stats_new['country'] 列转成字符串，再将 region + country + seller_name 用下划线连接，放到 stats_new 中的一个新列名叫: unique_id


In [11]:

#读取 whole_seller_stats.xlsx 到stats
#在 pd.read_excel 函数中指定 na_values = [''], keep_default_na = False 这两个参数

#   na_values = [''] 表示只把 ‘’ (空值) 当成空值/缺失值
   
#   keep_default_na = False 表示不把 NA 等字符串当成空值 

#   如大家不加上面两个参数，read_excel 可能会把 NA 当成空值

stats = pd.read_excel("whole_seller_stats.xlsx", na_values = [''], keep_default_na = False)
stats

,region,country,seller_name
0,NA,US,Bob
1,NA,US,Tom
2,NA,404,John
3,NA,CA,Hanks
4,NA,US,NaN


In [25]:
#* 把 stats 空的数据去掉(比如没有区域、卖家名信息的行), 放到 stats_new 中
stats_new = stats.dropna().copy()
stats_new

,region,country,seller_name
0,NA,US,Bob
1,NA,US,Tom
2,NA,404,John
3,NA,CA,Hanks


In [26]:
# 查看 stats_new 里各列的数据类型  
stats_new.dtypes

region         object
country        object
seller_name    object
dtype: object

In [27]:
#查看 seller_name == "John" 这一行 country 的数据类型 (第3行，第2列）
print(stats_new.iloc[2,1])
print(type(stats_new.iloc[2,1]))


404
<class 'int'>


In [32]:
#* 查看 seller_name == "Tom" 这一行 country 的数据类型 (第2行，第2列）
print(stats_new.iloc[1,1])
print(type(stats_new.iloc[1,1]))


US
<class 'str'>


In [29]:
#* 请尝试执行下面代码，看有什么问题
# stats_new['region'] + "_" + stats_new['country'] + "_" + stats['seller_name']
stats_new["unique_id"] = stats_new['region'] + "_" + stats_new['country'] + "_" + stats['seller_name']
stats_new

TypeError: can only concatenate str (not "int") to str

In [30]:
# 请将 stats_new['country'] 列转成字符串，再将 region + country + seller_name 用下划线连接，放到 stats_new 中的一个新列名叫: unique_id
stats_new["unique_id"] = stats_new['region'] + "_" + stats_new['country'].astype(str) + "_" + stats['seller_name']
stats_new

,region,country,seller_name,unique_id
0,NA,US,Bob,NA_US_Bob
1,NA,US,Tom,NA_US_Tom
2,NA,404,John,NA_404_John
3,NA,CA,Hanks,NA_CA_Hanks


# 怎么处理客户错填的销量信息
---
备注：这里综合了 p134面的内容

做为一个数据分析师，从战术层面看，TA的最大工作就清洗数据；你一定要尽早把无效数据识别出来，标注出来！

而且一定要要用系统认可的空值  NaN 标名；如果是程序员则会用 None(Python)/null(C) 来标明

这里转换数类型时，使用了 pd.to_numeric 函数，[pd.to_numeric 详细的用法](https://blog.csdn.net/jinruoyanxu/article/details/79150896)

---
背景：

* 你现在负责公司全球线下销售数据的收集，由经销商发邮件给你

* 经销商发过来的销量数据千奇百怪，销量数据(quantity) 很不规范

* 公司张总要你写个程序自动分析出是哪些经销商填写的不规范，好发邮件通报批评

---
* 读取 sales_data.csv（线下销售数据) 到 sales

* 查看 sales 的数据类型

  你可以看到，由于有经销商没有按规范填写，所以销量(quantity)字段并不是数字类型
  
* 使用 pd.to_numeric 将 sales 中的 quantity 字段转成数字，放到 temp_quantity

  转换不成功的数字转换成空值(NaN), 参数：errors="coerce"
  
* 查看 temp_quantity 里哪些行的数据是空值 (isnull())

* 利用 temp_quantity 从 sales 中提取未正常填写的数据，放到 report 中

* 将 report 保存到 report.xlsx 文件中
  


In [3]:
#读取 sales_data.csv（线下销售数据) 到 sales
sales = pd.read_csv("sales_data.csv")
sales

,seller_name,sku,quantity
0,Tom,A1234,100
1,Tom,A1235,200
2,Tom,A1236,300
3,Tom,A1237,missing
4,李老板,A1234,200
5,李老板,A1235,100
6,李老板,A1237,断货


In [4]:
# 查看 sales 的数据类型
#  你可以看到，由于有经销商没有按规范填写，所以销量(quantity)字段并不是数字类型

sales.dtypes

seller_name    object
sku            object
quantity       object
dtype: object

In [7]:
#使用 pd.to_numeric 将 sales 中的 quantity 字段转成数字，放到 temp_quantity
# 转换不成功的数字转换成空值(NaN), 参数：errors="coerce"

temp_quantity = pd.to_numeric(sales['quantity'],errors="coerce")
temp_quantity

0    100.0
1    200.0
2    300.0
3      NaN
4    200.0
5    100.0
6      NaN
Name: quantity, dtype: float64

In [8]:
#* 查看 temp_quantity 里哪些行的数据是空值 (isnull())
temp_quantity.isnull()

0    False
1    False
2    False
3     True
4    False
5    False
6     True
Name: quantity, dtype: bool

In [11]:
#* 利用 temp_quantity 从 sales 中提取未正常填写的数据，放到 report 中
#  使用temp_quantity.isnull() 从 sales 中过滤
report = sales[temp_quantity.isnull()]
report

,seller_name,sku,quantity
3,Tom,A1237,missing
6,李老板,A1237,断货


In [12]:
#* 将 report 保存到 report.xlsx 文件中
report.to_excel("report.xlsx")

# 6. 参考资料
* [dropna滤除缺失数据](https://blog.csdn.net/weixin_38168620/article/details/79596798)
* [pd.read_csv/pd.read_excel 函数中 na_values 参数的意义和用法](https://blog.csdn.net/weixin_44520259/article/details/106053987)
* [pd.to_numeric 详细的用法](https://blog.csdn.net/jinruoyanxu/article/details/79150896)